In [1]:
import numpy as np
import pandas as pnd

dataset = 1
match dataset:
    case 1:
        test = 'fashion-mnist_test.csv'
        train = 'fashion-mnist_train.csv'
    case 2: 
        test = "mnist_test.csv"
        train = "mnist_train.csv"
testdata = pnd.read_csv(test)
traindata = pnd.read_csv(train)

testdata_pixel = testdata.drop(testdata.columns[0], axis=1).to_numpy()
traindata_pixel = traindata.drop(traindata.columns[0], axis=1).to_numpy()
label_train = traindata[traindata.columns[0]].to_numpy()
label_test = testdata[testdata.columns[0]].to_numpy()

class_names = np.array(["T-shirt / Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"])

In [4]:
from scipy import spatial
from collections import Counter
from scipy.stats import mode
from functions.PCA import pca
from pathlib import Path  
filepath = Path('C:/Users/awark')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

PCs_train, PCs_test = pca(traindata_pixel,testdata_pixel,20)


def knn_kdtree(PCs_train, PCs_test, k, label_train, label_test, testsize):
    """returns the accuray of the KNN for test images

    Args:
        PCs_train (numpy array): transformed training data
        PCs_test (numpy array): transformed testing data
        k (int): number of neighbours 
        label_train (numpy array 1D): labels of training data
        label_test (numpy array 1D): labels of testing data
        testsize (int): number of testing images you want to classify
        
    Returns:
        knn for number of testsize images
    """

    result = np.array([])
    kd_tree = spatial.KDTree(PCs_train,leafsize=10)
    count_count = 0
    for i in range(0, len(label_test), testsize):
        if count_count*testsize % 1000 == 0:
            print("Current progress is " + str(count_count*100/(len(label_test)/testsize)) + " percent.")
        count_count += 1
        dist, neighbour_index = kd_tree.query(PCs_test[i:i+testsize, None],p=2,k=k, workers = -1)
        neighbour_label = label_train[neighbour_index]
        batch_result = [mode(neighbour_label,axis=2)[0]]
        batch_result = np.array(batch_result)
        batch_result = np.squeeze(batch_result)
        batch_result = batch_result.astype(int)
        result = result.astype(int)
        result = np.concatenate((result, batch_result), axis=0)
    return result

#asdf = knn_quality2(traindata_pixel, testdata_pixel, 3, label_train, label_test, 40)


(784, 784)
Our eigenvectors explain 70.02936636789705 % of total variance


In [5]:
total_var_results=[]
for k in range(2, 16): 
    new_column = knn_kdtree(PCs_train, PCs_test, k, label_train, label_test, 40).tolist()
    total_var_results.append(new_column)
    print("Batch " + str(k-1) + " is now finished.")
total_var_results = pnd.DataFrame(total_var_results).transpose()


    

    

Current progress is 0.0 percent.
Current progress is 10.0 percent.
Current progress is 20.0 percent.
Current progress is 30.0 percent.
Current progress is 40.0 percent.
Current progress is 50.0 percent.
Current progress is 60.0 percent.
Current progress is 70.0 percent.
Current progress is 80.0 percent.
Current progress is 90.0 percent.
Batch 1 is now finished.
Current progress is 0.0 percent.
Current progress is 10.0 percent.
Current progress is 20.0 percent.
Current progress is 30.0 percent.
Current progress is 40.0 percent.
Current progress is 50.0 percent.
Current progress is 60.0 percent.
Current progress is 70.0 percent.
Current progress is 80.0 percent.
Current progress is 90.0 percent.
Batch 2 is now finished.
Current progress is 0.0 percent.
Current progress is 10.0 percent.
Current progress is 20.0 percent.
Current progress is 30.0 percent.
Current progress is 40.0 percent.
Current progress is 50.0 percent.
Current progress is 60.0 percent.
Current progress is 70.0 percent.
C

In [24]:
for i in range(14):
    accuracy = np.sum(total_var_results.iloc[:,i]==label_test)/len(label_test)
    print(accuracy)


0.8194
0.8353
0.8411
0.8454
0.8456
0.8468
0.8451
0.8456
0.8468
0.8436
0.8427
0.8423
0.8422
0.8436
